In [1]:
import pandas as pd
import numpy as np
import pulp
from scipy.optimize import linprog

In [2]:
cps = pd.read_csv('../cps_data/cps_raw.csv.gz', compression='gzip')
cps = cps.fillna(0.)
stage_1_factors = pd.read_csv('../cps_stage1/stage_1_factors.csv', index_col=0)
stage_2_targets = pd.read_csv('../cps_stage1/stage_2_targets.csv', index_col=0)

In [3]:
cps['MARS'] = np.where(cps.JS == 3, 4, cps.JS)
cps['e00100'] = cps['JCPS9'] + cps['JCPS19']

In [4]:
weights = pd.DataFrame()
weights['WT2014'] = cps.WT * 100

In [5]:
def solve_lp_for_year(data, factors, targets, year, tol):
    """
    Parameters
    ----------
    data: CPS data
    factors: growth factors
    targets: aggregate targets
    year: year LP model is being solved for
    tol: tolerance
    """
    def target(target_val, pop, factor, value):
        return (target_val * pop / factor * 1000 - value)
    

    print('Preparing Coefficient Matrix for {}'.format(year))
    data_len = len(data.WT)
    # dictionary used just to play with the targets we add
    lhs_vars = {}

    # don't divide by 100 because WT hasn't been multiplied by 100
    s006 = np.where(data.SS > 0,
                    data.WT * factors['APOPSNR'][year],
                    data.WT * factors['ARETS'][year])

    single_returns = np.where((data.JS == 1) & (data.FILST == 1),
                             s006, 0)
    joint_returns = np.where((data.JS == 2) & (data.FILST == 1),
                            s006, 0)
    hh_returns = np.where((data.JS == 3) & (data.FILST == 1),
                         s006, 0)
    returns_w_ss = np.where((data.SS > 0) & (data.FILST == 1),
                            s006, 0)
    dep_exemptions = np.where(data.JS == 2, data.XXTOT - 2, data.XXTOT - 1) * s006
    interest = data.INTST * s006
    dividend = data.DBE * s006
    biz = data.JCPS25 + data.JCPS35
    biz_income = np.where(biz > 0, biz, 0) * s006
    biz_loss = np.where(biz < 0, -biz, 0) * s006
    cap_gain = np.where(data.CGAGIX > 0, data.CGAGIX, 0) * s006
    pension = data.PENSIONS * s006
    sch_e_income = np.where(data.RENTS > 0, data.RENTS, 0) * s006
    sch_e_loss = np.where(data.RENTS < 0, data.RENTS, 0) * s006
    ss_income = np.where(data.FILST == 1, data.SS, 0) * s006
    ucomp = data.UCOMP * s006

    # wage distribution
    wage1 = np.where((data.e00100 <= 10000), data.WAS, 0) * s006
    wage2 = np.where((data.e00100 > 10000) & (data.e00100 <= 20000),
                     data.WAS, 0) * s006
    wage3 = np.where((data.e00100 > 20000) & (data.e00100 <= 30000),
                     data.WAS, 0) * s006
    wage4 = np.where((data.e00100 > 30000) & (data.e00100 <= 40000),
                     data.WAS, 0) * s006
    wage5 = np.where((data.e00100 > 40000) & (data.e00100 <= 50000),
                     data.WAS, 0) * s006
    wage6 = np.where((data.e00100 > 50000) & (data.e00100 <= 75000),
                     data.WAS, 0) * s006
    wage7 = np.where((data.e00100 > 75000) & (data.e00100 <= 100000),
                     data.WAS, 0) * s006
    wage8 = np.where((data.e00100 > 100000), data.WAS, 0) * s006

    lhs_vars['single_returns'] = single_returns
    lhs_vars['joint_returns'] = joint_returns
    lhs_vars['hh_returns'] = hh_returns
    lhs_vars['returns_w_ss'] = returns_w_ss
    lhs_vars['dep_exemptions'] = dep_exemptions
    lhs_vars['interest'] = interest
    lhs_vars['dividend'] = dividend
    lhs_vars['biz_income'] = biz_income
    lhs_vars['biz_loss'] = biz_loss
    lhs_vars['cap_gain'] = cap_gain
    lhs_vars['pension'] = pension
    lhs_vars['sch_e_income'] = sch_e_income
    lhs_vars['sch_e_loss'] = sch_e_loss
    lhs_vars['ss_income'] = ss_income
    lhs_vars['ucomp'] = ucomp
    lhs_vars['wage1'] = wage1
    lhs_vars['wage2'] = wage2
    lhs_vars['wage3'] = wage3
    lhs_vars['wage4'] = wage4
    lhs_vars['wage5'] = wage5
    lhs_vars['wage6'] = wage6
    lhs_vars['wage7'] = wage7
    lhs_vars['wage8'] = wage8
    
    print('Preparing Targets for {}'.format(year))
    apopn = factors['APOPN'][year]
    aints = factors['AINTS'][year]
    adivs = factors['ADIVS'][year]
    aschci = factors['ASCHCI'][year]
    aschcl = factors['ASCHCL'][year]
    acgns = factors['ACGNS'][year]
    atxpy = factors['ATXPY'][year]
    aschei = factors['ASCHEI'][year]
    aschel = factors['ASCHEL'][year]
    asocsec = factors['ASOCSEC'][year]
    apopsnr = factors['APOPSNR'][year]
    aucomp = factors['AUCOMP'][year]
    awage = factors['AWAGE'][year]
    
    year = str(year)
    rhs_vars = {}

    rhs_vars['single_returns'] = targets[year]['Single'] - single_returns.sum()
    rhs_vars['joint_returns'] = targets[year]['Joint'] - joint_returns.sum()
    rhs_vars['hh_returns'] = targets[year]['HH'] - hh_returns.sum()
    target_name = 'SS_return'
    rhs_vars['returns_w_ss'] = targets[year][target_name] - returns_w_ss.sum()
    target_name = 'Dep_return'
    rhs_vars['dep_exemptions'] = targets[year][target_name] - dep_exemptions.sum()
    rhs_vars['interest'] = target(targets[year]['INTS'], apopn, aints, interest.sum())
    rhs_vars['dividend'] = target(targets[year]['DIVS'], apopn, adivs, dividend.sum())
    rhs_vars['biz_income'] = target(targets[year]['SCHCI'], apopn, aschci, biz_income.sum())
    rhs_vars['biz_loss'] = target(targets[year]['SCHCL'], apopn, aschcl, biz_loss.sum())
    rhs_vars['cap_gain'] = target(targets[year]['CGNS'], apopn, acgns, cap_gain.sum())
    rhs_vars['pension'] = target(targets[year]['Pension'], apopn, atxpy, pension.sum())
    rhs_vars['sch_e_income'] = target(targets[year]['SCHEI'], apopn, aschei, sch_e_income.sum())
    rhs_vars['sch_e_loss'] = target(targets[year]['SCHEL'], apopn, aschel, sch_e_loss.sum())
    rhs_vars['ss_income'] = target(targets[year]['SS'], apopsnr, asocsec, ss_income.sum())
    rhs_vars['ucomp'] = target(targets[year]['UCOMP'], apopn, aucomp, ucomp.sum())
    rhs_vars['wage1'] = target(targets[year]['wage1'], apopn, awage, wage1.sum())
    rhs_vars['wage2'] = target(targets[year]['wage2'], apopn, awage, wage2.sum())
    rhs_vars['wage3'] = target(targets[year]['wage3'], apopn, awage, wage3.sum())
    rhs_vars['wage4'] = target(targets[year]['wage4'], apopn, awage, wage4.sum())
    rhs_vars['wage5'] = target(targets[year]['wage5'], apopn, awage, wage5.sum())
    rhs_vars['wage6'] = target(targets[year]['wage6'], apopn, awage, wage6.sum())
    rhs_vars['wage7'] = target(targets[year]['wage7'], apopn, awage, wage7.sum())
    rhs_vars['wage8'] = target(targets[year]['wage8'], apopn, awage, wage8.sum())
    
    """model_vars = ['single_returns', 'joint_returns', 'hh_returns', 'returns_w_ss',
                  'cap_gain',
                  'wage1', 'wage2', 'wage3', 'wage4', 'wage5', 'wage6', 'wage7']"""
    model_vars = ['single_returns', 'joint_returns', 'returns_w_ss', 'dep_exemptions',
                  'interest', 'dividend', 'biz_income', 'pension', 'ss_income',
                  'ucomp', 'wage1', 'wage2', 'wage3', 'wage4', 'wage5', 'wage6',
                  'wage7', 'wage8']
    vstack_vars = []
    b = []  # list to hold the targets
    for var in model_vars:
        vstack_vars.append(lhs_vars[var])
        b.append(rhs_vars[var])
        print(var, rhs_vars[var])
    vstack_vars = tuple(vstack_vars)
    one_half_lhs = np.vstack(vstack_vars)
    
    # coefficients for r and s
    a1 = np.array(one_half_lhs)
    a2 = np.array(-one_half_lhs)

    # set up LP model
    print('Constructing LP Model')
    LP = pulp.LpProblem('CPS Stage 2', pulp.LpMinimize)
    r = pulp.LpVariable.dicts('r', data.index, lowBound=0)
    s = pulp.LpVariable.dicts('s', data.index, lowBound=0)
    # add objective function
    LP += pulp.lpSum([r[i] + s[i] for i in data.index])
    # add constrains
    for i in data.index:
        LP += r[i] + s[i] <= tol
    for i in range(len(b)):
        LP += pulp.lpSum([(a1[i][j] * r[j] + a2[i][j] * s[j]) for j in data.index]) == b[i]
    print('Solving Model...')
    pulp.LpSolverDefault.msg = 1
    LP.solve()
    print(pulp.LpStatus[LP.status])
    
    # apply r and s to the weights
    r_val = np.array([r[i].varValue for i in r])
    s_val = np.array([s[i].varValue for i in s])
    z = (1. + r_val - s_val) * s006 * 100

    return z

In [7]:
weights['WT2015'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2015, .50)

Preparing Coefficient Matrix for 2015
Preparing Targets for 2015
single_returns -9372706.25204
joint_returns -9011846.50149
returns_w_ss -8218752.30683
dep_exemptions 5435712.44091
interest -7121546511.12
dividend 1218840953.07
biz_income 68663163221.9
pension -45985770574.1
ss_income -110803008111.0
ucomp -9359381411.55
wage1 -17080409158.7
wage2 43014545738.0
wage3 30810671390.8
wage4 35484664370.2
wage5 16974544009.1
wage6 -70205615031.0
wage7 -63280884379.5
wage8 118005919318.0
Constructing LP Model
Solving Model...
Optimal


In [8]:
weights['WT2016'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2016, .50)

Preparing Coefficient Matrix for 2016
Preparing Targets for 2016
single_returns -9834330.49767
joint_returns -9455045.09381
returns_w_ss -8491988.42225
dep_exemptions 4615844.79943
interest -9563538925.17
dividend -2833655272.74
biz_income 66020483570.3
pension -61668221905.3
ss_income -114486703932.0
ucomp -9866438769.09
wage1 -18400552362.8
wage2 41317915105.7
wage3 27943775944.4
wage4 32351100167.7
wage5 13541225488.3
wage6 -79818573670.2
wage7 -72154903670.4
wage8 88696520078.9
Constructing LP Model
Solving Model...
Optimal


In [9]:
weights['WT2017'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2017, .50)

Preparing Coefficient Matrix for 2017
Preparing Targets for 2017
single_returns -10297857.4633
joint_returns -9900102.83071
returns_w_ss -8772876.37567
dep_exemptions 3645309.81211
interest -12236543309.3
dividend -7227717867.11
biz_income 62887812079.8
pension -78126856357.9
ss_income -118273559772.0
ucomp -10442420662.1
wage1 -19939755963.9
wage2 39259534951.0
wage3 24490903429.9
wage4 28577640572.8
wage5 9436812815.43
wage6 -91069090927.4
wage7 -82525288604.6
wage8 53879767785.0
Constructing LP Model
Solving Model...
Optimal


In [10]:
weights['WT2018'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2018, .50)

Preparing Coefficient Matrix for 2018
Preparing Targets for 2018
single_returns -10801491.5237
joint_returns -10383614.4482
returns_w_ss -9066951.55241
dep_exemptions 2768922.02938
interest -14881878189.0
dividend -11639855228.6
biz_income 60018864632.0
pension -95295557678.4
ss_income -122238202212.0
ucomp -10979385031.7
wage1 -21334089007.2
wage2 37430624985.0
wage3 21431035150.1
wage4 25231406064.5
wage5 5782174799.56
wage6 -101259323797.0
wage7 -91937468368.9
wage8 22648209032.5
Constructing LP Model
Solving Model...
Optimal


In [11]:
weights['WT2019'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2019, .50)

Preparing Coefficient Matrix for 2019
Preparing Targets for 2019
single_returns -11351775.537
joint_returns -10911844.3867
returns_w_ss -9374607.27584
dep_exemptions 2040023.3623
interest -17438243234.9
dividend -15989955222.0
biz_income 57579176560.4
pension -113110078407.0
ss_income -126385933929.0
ucomp -11453544208.7
wage1 -22504370042.5
wage2 35959606090.3
wage3 18976290048.6
wage4 22543674762.3
wage5 2820343420.46
wage6 -109801862027.0
wage7 -99856571370.1
wage8 -3022237565.42
Constructing LP Model
Solving Model...
Optimal


In [12]:
weights['WT2020'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2020, .50)

Preparing Coefficient Matrix for 2020
Preparing Targets for 2020
single_returns -11951399.1944
joint_returns -11487379.5528
returns_w_ss -9698357.61575
dep_exemptions 1427196.17162
interest -19960491119.8
dividend -20362851910.4
biz_income 55463119961.9
pension -131802224082.0
ss_income -130750648936.0
ucomp -11879298326.1
wage1 -23495350707.6
wage2 34770708330.4
wage3 17005905112.7
wage4 20382422714.5
wage5 413325816.894
wage6 -117033631471.0
wage7 -106591124806.0
wage8 -24269946646.2
Constructing LP Model
Solving Model...
Optimal


In [13]:
weights['WT2021'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2021, .50)

Preparing Coefficient Matrix for 2021
Preparing Targets for 2021
single_returns -12537033.4407
joint_returns -12049496.6039
returns_w_ss -10016430.2921
dep_exemptions 801297.178716
interest -22463255028.2
dividend -24687773246.0
biz_income 53316576768.3
pension -150160386467.0
ss_income -135038819213.0
ucomp -12309188989.7
wage1 -24506958010.9
wage2 33548036505.4
wage3 14975332469.2
wage4 18156015122.2
wage5 -2061754302.11
wage6 -124414429315.0
wage7 -113458341229.0
wage8 -46040037881.8
Constructing LP Model
Solving Model...
Optimal


In [14]:
weights['WT2022'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2022, .50)

Preparing Coefficient Matrix for 2022
Preparing Targets for 2022
single_returns -13156530.3046
joint_returns -12644091.8862
returns_w_ss -10347941.2389
dep_exemptions 198564.636908
interest -25027635664.5
dividend -29154004194.9
biz_income 51206846431.0
pension -169398835208.0
ss_income -139508160633.0
ucomp -12731218486.7
wage1 -25476769489.8
wage2 32382178238.9
wage3 13056864145.5
wage4 16050293160.8
wage5 -4407681909.62
wage6 -131505067543.0
wage7 -120069418625.0
wage8 -66837982900.9
Constructing LP Model
Solving Model...
Optimal


In [15]:
weights['WT2023'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2023, .50)

Preparing Coefficient Matrix for 2023
Preparing Targets for 2023
single_returns -13765958.8459
joint_returns -13229030.0764
returns_w_ss -10675378.5714
dep_exemptions -416847.945683
interest -27583240448.6
dividend -33594185438.1
biz_income 49062700356.8
pension -188418729796.0
ss_income -143922582680.0
ucomp -13157380517.7
wage1 -26465363263.2
wage2 31183251054.4
wage3 11082216096.2
wage4 13883563988.9
wage5 -6816761239.9
wage6 -138734307702.0
wage7 -126804708580.0
wage8 -88128133019.0
Constructing LP Model
Solving Model...
Optimal


In [16]:
weights['WT2024'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2024, .50)

Preparing Coefficient Matrix for 2024
Preparing Targets for 2024
single_returns -14357448.488
joint_returns -13796760.0146
returns_w_ss -10995144.8211
dep_exemptions -1041654.03514
interest -30102970506.2
dividend -37957204743.9
biz_income 46902543871.0
pension -206977661072.0
ss_income -148233585255.0
ucomp -13585343844.3
wage1 -27469328435.5
wage2 29958337023.6
wage3 9059061476.1
wage4 11664562334.1
wage5 -9280044874.86
wage6 -146072695539.0
wage7 -133635293616.0
wage8 -109817000956.0
Constructing LP Model
Solving Model...
Optimal


In [17]:
weights['WT2025'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2025, .50)

Preparing Coefficient Matrix for 2025
Preparing Targets for 2025
single_returns -14974295.123
joint_returns -14388820.6815
returns_w_ss -11327068.6792
dep_exemptions -1699974.86043
interest -32745544663.7
dividend -42539735721.3
biz_income 44602313100.3
pension -226448689171.0
ss_income -152708493442.0
ucomp -14030105188.5
wage1 -28514794222.3
wage2 28652404263.4
wage3 6921453096.54
wage4 9319198659.63
wage5 -11872961830.7
wage6 -153740630231.0
wage7 -140773744439.0
wage8 -132633907103.0
Constructing LP Model
Solving Model...
Optimal


In [18]:
weights['WT2026'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2026, .50)

Preparing Coefficient Matrix for 2026
Preparing Targets for 2026
single_returns -15551225.1305
joint_returns -14942587.2821
returns_w_ss -11641271.3623
dep_exemptions -2358770.95347
interest -35277003052.6
dividend -46903103639.1
biz_income 42336402339.2
pension -244783832351.0
ss_income -156944489509.0
ucomp -14470235740.9
wage1 -29566384601.6
wage2 27339529975.2
wage3 4755741829.49
wage4 6944828084.41
wage5 -14496201985.4
wage6 -161437771956.0
wage7 -147928861079.0
wage8 -155597537057.0
Constructing LP Model
Solving Model...
Optimal


In [19]:
weights['WT2027'] = solve_lp_for_year(cps, stage_1_factors, stage_2_targets, 2027, .50)

Preparing Coefficient Matrix for 2027
Preparing Targets for 2027
single_returns -16081499.0445
joint_returns -15451593.8675
returns_w_ss -11934673.4745
dep_exemptions -3016115.33459
interest -37675570044.3
dividend -51005863335.3
biz_income 40117156702.6
pension -261781542156.0
ss_income -160900057867.0
ucomp -14904046695.5
wage1 -30622231950.9
wage2 26023387057.3
wage3 2565008654.31
wage4 4545099691.02
wage5 -17145937357.0
wage6 -169147324255.0
wage7 -155083606911.0
wage8 -178659296508.0
Constructing LP Model
Solving Model...
Optimal


In [20]:
weights.to_csv('cps_weights.csv.gz', index=False, compression='gzip')